In [1]:
import requests
import json
import sys

sys.path.append('..')

# Data Manipulation
import pandas as pd

from utils import *

# Agregados

In [2]:
url = 'https://servicodados.ibge.gov.br/api/v3/agregados'
df_aggregated = make_df(get_request_json(url))
df_aggregated

,id,nome,agregados
0,D5,Áreas Urbanizadas,"[{'id': '8418', 'nome': 'Áreas urbanizadas, Lo..."
1,CL,Cadastro Central de Empresas,"[{'id': '1732', 'nome': 'Dados gerais das empr..."
2,CA,Censo Agropecuário,"[{'id': '1278', 'nome': 'Agroindústria rural n..."
3,ME,"Censo Comum do Mercosul, Bolívia e Chile","[{'id': '2059', 'nome': 'Domicílios e Populaçã..."
4,CD,Censo Demográfico,"[{'id': '1301', 'nome': 'Área e Densidade demo..."
...,...,...,...
62,VS,Produção da Extração Vegetal e da Silvicultura,"[{'id': '5930', 'nome': 'Área total existente ..."
63,PO,Produção de Ovos de Galinha,"[{'id': '915', 'nome': 'Número de informantes,..."
64,IO,Produto Interno Bruto dos Municípios,"[{'id': '599', 'nome': 'Índice de Gini do prod..."
65,XE,Projeção da População,"[{'id': '7362', 'nome': 'Esperança de vida ao ..."


In [3]:
research_ids = df_aggregated['id'].unique()
print(list(research_ids))

['D5', 'CL', 'CA', 'ME', 'CD', 'CM', 'DU', 'DT', 'C4', 'DR', 'DZ', 'SU', 'ST', 'DE', 'XF', 'FP', 'IU', 'IL', 'IR', 'IJ', 'PC', 'IA', 'IQ', 'IG', 'PG', 'LA', 'C2', 'AC', 'PB', 'PY', 'PP', 'AM', 'ES', 'PM', 'IT', 'OF', 'SH', 'SE', 'PT', 'RC', 'PI', 'PK', 'PJ', 'DG', 'PZ', 'XY', 'AB', 'MC', 'PE', 'SC', 'PL', 'SB', 'XN', 'AA', 'PD', 'B5', 'BB', 'DD', 'AX', 'CQ', 'LT', 'PA', 'VS', 'PO', 'IO', 'XE', 'SI']


In [4]:
%%time
df_researches = pd.DataFrame()

research_ids = df_aggregated['id'].unique()

for research_id in research_ids:
    # get just the row for the given research id
    df_research = df_aggregated[df_aggregated['id'] == research_id]
    # explode the row to get all the agregados 
    df_research = df_research.explode('agregados')
    # reset index
    df_research = df_research.reset_index()
    # normalize the agregados column and rename the columns
    df_normalize = pd.json_normalize(df_research['agregados'])
    df_normalize = df_normalize.rename(columns = {'id': 'id-agregado', 'nome': 'nome-agregado'})
    # concat the agregado data to df_research
    df_research = pd.concat([df_research, df_normalize], axis = 1)
    # delete the dict column
    df_research = df_research.drop('agregados', axis = 1)
    # rename the columns
    df_research = df_research.rename(columns = {'id': 'id-pesquisa', 'nome': 'nome-pesquisa'})
    # concat the research data to df_researches
    df_researches = pd.concat([df_researches, df_research], axis = 0)
# reset index
df_researches = df_researches.reset_index(drop = True)

CPU times: total: 188 ms
Wall time: 440 ms


In [5]:
df_researches

,index,id-pesquisa,nome-pesquisa,id-agregado,nome-agregado
0,0,D5,Áreas Urbanizadas,8418,"Áreas urbanizadas, Loteamento vazio, Área tota..."
1,1,CL,Cadastro Central de Empresas,1732,Dados gerais das empresas por faixas de pessoa...
2,1,CL,Cadastro Central de Empresas,1735,Dados gerais das unidades locais por faixas de...
3,1,CL,Cadastro Central de Empresas,1734,Dados gerais das unidades locais por faixas de...
4,1,CL,Cadastro Central de Empresas,1733,Dados gerais das unidades locais por faixas de...
...,...,...,...,...,...
8095,66,SI,Sistema Nacional de Pesquisa de Custos e Índic...,6586,Custo médio m² em moeda corrente e variações p...
8096,66,SI,Sistema Nacional de Pesquisa de Custos e Índic...,32,"Custo médio m² em variação percentual, por tip..."
8097,66,SI,Sistema Nacional de Pesquisa de Custos e Índic...,34,"Preços medianos, por materiais e serviços (sér..."
8098,66,SI,Sistema Nacional de Pesquisa de Custos e Índic...,2062,"Preços medianos, por materiais e serviços (sér..."


# Metadados das pesquisas

In [43]:
%%time
df_metadata = pd.DataFrame()

aggregate_ids = df_researches['id-agregado'].unique()[0:10]

for aggregate_id in aggregate_ids:
    # create the url to request the metadata
    url_metadata = f"https://servicodados.ibge.gov.br/api/v3/agregados/{aggregate_id}/metadados"
    # make the request and get the metadata json
    df_aux = make_df([get_request_json(url_metadata)])
    # explode the row to get all the metadata
    # df_aux = df_aux.explode(['metadados'])
    # concat the metadata to df_metadata
    df_metadata = pd.concat([df_metadata, df_aux], axis = 0)
# reset index
df_metadata = df_metadata.reset_index(drop = True)

CPU times: total: 219 ms
Wall time: 2.27 s


In [44]:
df_metadata.info()
df_metadata.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 13 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   id                               10 non-null     int64 
 1   nome                             10 non-null     object
 2   URL                              10 non-null     object
 3   pesquisa                         10 non-null     object
 4   assunto                          10 non-null     object
 5   periodicidade_frequencia         10 non-null     object
 6   periodicidade_inicio             10 non-null     int64 
 7   periodicidade_fim                10 non-null     int64 
 8   nivelTerritorial_Administrativo  10 non-null     object
 9   nivelTerritorial_Especial        10 non-null     object
 10  nivelTerritorial_IBGE            10 non-null     object
 11  variaveis                        10 non-null     object
 12  classificacoes                   10 non

,id,nome,URL,pesquisa,assunto,periodicidade_frequencia,periodicidade_inicio,periodicidade_fim,nivelTerritorial_Administrativo,nivelTerritorial_Especial,nivelTerritorial_IBGE,variaveis,classificacoes
0,8418,"Áreas urbanizadas, Loteamento vazio, Área tota...",https://sidra.ibge.gov.br/tabela/8418,Áreas Urbanizadas,Geral,anual,2019,2019,"[N2, N6, N3]","[N23, N132]",[],"[{'id': 12747, 'nome': 'Áreas urbanizadas dens...",[]
1,1732,Dados gerais das empresas por faixas de pessoa...,https://sidra.ibge.gov.br/tabela/1732,Cadastro Central de Empresas,Empresas e outras organizações,anual,1996,2006,"[N1, N2, N3]",[],[],"[{'id': 630, 'nome': 'Número de empresas', 'un...","[{'id': 11939, 'nome': 'Classificação Nacional..."
2,1735,Dados gerais das unidades locais por faixas de...,https://sidra.ibge.gov.br/tabela/1735,Cadastro Central de Empresas,Unidades locais,anual,1996,2006,"[N1, N2, N6, N3]",[],[],"[{'id': 706, 'nome': 'Número de unidades locai...","[{'id': 319, 'nome': 'Faixas de pessoal ocupad..."
3,1734,Dados gerais das unidades locais por faixas de...,https://sidra.ibge.gov.br/tabela/1734,Cadastro Central de Empresas,Unidades locais,anual,1996,2006,"[N1, N2, N6, N3]",[],[],"[{'id': 706, 'nome': 'Número de unidades locai...","[{'id': 319, 'nome': 'Faixas de pessoal ocupad..."
4,1733,Dados gerais das unidades locais por faixas de...,https://sidra.ibge.gov.br/tabela/1733,Cadastro Central de Empresas,Unidades locais,anual,1996,2006,"[N1, N2, N3]",[],[],"[{'id': 706, 'nome': 'Número de unidades locai...","[{'id': 11939, 'nome': 'Classificação Nacional..."


In [45]:
df_metadata['classificacoes'][1]

[{'id': 11939,
  'nome': 'Classificação Nacional de Atividades Econômicas (CNAE)',
  'sumarizacao': {'status': True, 'excecao': []},
  'categorias': [{'id': 0, 'nome': 'Total', 'unidade': None, 'nivel': 0},
   {'id': 96897,
    'nome': 'A Agricultura, pecuária, silvicultura e exploração florestal',
    'unidade': None,
    'nivel': 1},
   {'id': 96898,
    'nome': '01 Agricultura, pecuária e serviços relacionados',
    'unidade': None,
    'nivel': 2},
   {'id': 96899,
    'nome': '01.1 Produção de lavouras temporárias',
    'unidade': None,
    'nivel': 3},
   {'id': 96900,
    'nome': '01.2 Horticultura e produtos de viveiro',
    'unidade': None,
    'nivel': 3},
   {'id': 96901,
    'nome': '01.3 Produção de lavouras permanentes',
    'unidade': None,
    'nivel': 3},
   {'id': 96903, 'nome': '01.4 Pecuária', 'unidade': None, 'nivel': 3},
   {'id': 96904,
    'nome': '01.5 Produção mista: lavoura e pecuária',
    'unidade': None,
    'nivel': 3},
   {'id': 96905,
    'nome': '01.6 

In [46]:
df_metadata['classificacoes'][2]

[{'id': 319,
  'nome': 'Faixas de pessoal ocupado',
  'sumarizacao': {'status': True, 'excecao': []},
  'categorias': [{'id': 0, 'nome': 'Total', 'unidade': None, 'nivel': 0},
   {'id': 7823, 'nome': '0 a 4', 'unidade': None, 'nivel': 1},
   {'id': 7824, 'nome': '5 a 9', 'unidade': None, 'nivel': 1},
   {'id': 7825, 'nome': '10 a 19', 'unidade': None, 'nivel': 1},
   {'id': 101226, 'nome': '20 a 29', 'unidade': None, 'nivel': 1},
   {'id': 96894, 'nome': '30 a 49', 'unidade': None, 'nivel': 1},
   {'id': 7827, 'nome': '50 a 99', 'unidade': None, 'nivel': 1},
   {'id': 7828, 'nome': '100 a 249', 'unidade': None, 'nivel': 1},
   {'id': 99734, 'nome': '250 a 499', 'unidade': None, 'nivel': 1},
   {'id': 7830, 'nome': '500 ou mais', 'unidade': None, 'nivel': 1}]},
 {'id': 11939,
  'nome': 'Classificação Nacional de Atividades Econômicas (CNAE)',
  'sumarizacao': {'status': True, 'excecao': []},
  'categorias': [{'id': 0, 'nome': 'Total', 'unidade': None, 'nivel': 0},
   {'id': 96897,
    '